In [1]:
from __future__ import print_function
import requests
import json
import csv

try:
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib

###
# go to terminal
# python get_bus_info_jl9200.py API_KEY Busline Busline.csv


In [2]:
MY_KEY ="22d5bcc9-a3c0-4858-94fb-2e1e8c04add9" # sys.argv[1]
BUS = "M5"  # sys.argv[2]
## input will be "python show_bus_locations.py <MTA_KEY> <BUS_LINE>"

url = "http://bustime.mta.info/api/siri/vehicle-monitoring.json?key="+ MY_KEY + "&VehicleMonitoringDetailLevel=calls&LineRef=" + BUS

print (url)

http://bustime.mta.info/api/siri/vehicle-monitoring.json?key=22d5bcc9-a3c0-4858-94fb-2e1e8c04add9&VehicleMonitoringDetailLevel=calls&LineRef=M5


In [3]:
response = urllib.urlopen(url)
data = response.read().decode("utf-8")
data = json.loads(data)
print (data)

{u'Siri': {u'ServiceDelivery': {u'SituationExchangeDelivery': [], u'ResponseTimestamp': u'2018-10-31T00:12:31.035-04:00', u'VehicleMonitoringDelivery': [{u'VehicleActivity': [{u'RecordedAtTime': u'2018-10-31T00:12:14.000-04:00', u'MonitoredVehicleJourney': {u'OriginRef': u'MTA_803083', u'Bearing': 53.823437, u'VehicleLocation': {u'Latitude': 40.759782, u'Longitude': -73.980488}, u'SituationRef': [], u'OperatorRef': u'MTA NYCT', u'VehicleRef': u'MTA NYCT_3943', u'BlockRef': u'MTA NYCT_MV_D8-Weekday-SDon_E_MV_26760_M5-215', u'DestinationName': u'WASHINGTON HEIGHTS  GW BRIDGE', u'DestinationRef': u'MTA_903231', u'JourneyPatternRef': u'MTA_M050416', u'MonitoredCall': {u'VisitNumber': 1, u'StopPointRef': u'MTA_403132', u'ExpectedDepartureTime': u'2018-10-31T00:12:32.035-04:00', u'StopPointName': u'AV OF THE AMERICAS/W 50 ST', u'Extensions': {u'Distances': {u'CallDistanceAlongRoute': 1490.89, u'StopsFromCall': 0, u'PresentableDistance': u'at stop', u'DistanceFromCall': 3.25}}, u'ExpectedArri

In [6]:
Num_bus = len(data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'])

In [7]:
# make bus_status_info function
# don't run
print(Num_bus)
# test print in notebook
for i in range(Num_bus):
    bus_lat = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['VehicleLocation']['Latitude']
    bus_lon = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['VehicleLocation']['Longitude']
    Stop_Name = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['OnwardCalls']['OnwardCall'][0]['StopPointName']
    Stop_Status = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['OnwardCalls']['OnwardCall'][0]['Extensions']['Distances']['PresentableDistance']
    print("%s,%s,%s,%s"%(bus_lat,bus_lon,Stop_Name,Stop_Status))


3
40.759782,-73.980488,AV OF THE AMERICAS/W 50 ST,at stop
40.766205,-73.977439,CENTRAL PK S/AVE OF THE AMERICAS,approaching
40.842207,-73.939254,BROADWAY/W 171 ST,approaching


In [8]:
filename = BUS +".csv"

# filename will be sys.argv[3]

def bus_status_info_csv():          
    with open(str(filename),'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Latitude','Longitude','Stop Name','Stop Status'])
        Num_bus = len(data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'])
        if Num_bus == 0:
            row = str('No Data')
            writer.writerow([row])
        else:
            for i in range(Num_bus):        
                bus_lat = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['VehicleLocation']['Latitude']
                bus_lon = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['VehicleLocation']['Longitude']
                checkEmpty = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['OnwardCalls']
                if len(checkEmpty) == 0:
                    Stop_Name = "N/A"
                    Stop_Status = "N/A"
                else:  
                    Stop_Name = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['OnwardCalls']['OnwardCall'][0]['StopPointName']
                    Stop_Status = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['OnwardCalls']['OnwardCall'][0]['Extensions']['Distances']['PresentableDistance']

                # need to convert as str
                row = str(bus_lat),str(bus_lon),str(Stop_Name),str(Stop_Status)
                writer.writerow(row)



In [9]:
# test incase the bus route didn't exist
try:  
    test = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']
    bus_status_info_csv()
except KeyError:
    print('No such route')  


